皮尔森相关系数（Pearson product-moment correlation coefficient）
$$\rho = \frac{\sum_{i=0}^{N-1} {(x_i - \mu_x)(y_i - \mu_y)}} {{N \sigma_x \sigma_y}}$$
$X$,$Y$是有$N$个测量值的向量，上式中的 $\mu_x$ 和 $\mu_y$ 是 $X$ 和 $Y$ 的均值， $\sigma_x$ 和 $\sigma_y$ 是它们的标准差。
$\rho$ 取值范围[-1,1] 

通常处理的信号为无偏（均值为0）归一化（标准差为1）的信号，所以：
$$\rho = \frac {1} {N} {\sum_{i=0}^{N-1} x_i y_i} $$
进一步：
$$r = {\sum_{i=0}^{N-1} x_i y_i} $$

如果把 $X$ 和 $Y$ 看做是向量，那么这个公式就是 点积（dot product） 的公式, 点积表征了信号之间的相似度
如果$X$ 和 $Y$ 都是归一化的：
$$X * Y = cos\theta$$

In [ ]:
import thinkdsp
import thinkplot
import numpy as np

def make_sine(offset):
    signal = thinkdsp.SinSignal(freq=440, offset=offset)
    wave = signal.make_wave(duration=0.5, framerate=10000)
    return wave

wave1 = make_sine(offset = 0)
#wave2 = make_sine(offset = 2 * np.pi) # max correlation
#wave2 = make_sine(offset = np.pi) # negtive max correlation
#wave2 = make_sine(offset = np.pi / 2) # min correlation
wave2 = make_sine(offset = 1) # offset = 1

wave1.plot(label="wave1")
wave2.plot(label="wave2")

thinkplot.config(xlabel='Time (s)',
                 ylabel='Amplitude',
                 xlim=[0, 0.01])

import numpy as np
corr_matrix = np.corrcoef(wave1.ys, wave2.ys)
print(corr_matrix)
print(wave1.corr(wave2))

In [ ]:
import matplotlib.pyplot as plt

#两个波形随着offset的不同，相关性发生的变化

wave_offsets = np.linspace(0, 2 * np.pi, 100)
corr_values = []
for i in range(len(wave_offsets)):
    wavei = make_sine(offset = wave_offsets[i])
    corr_values.append(wave1.corr(wavei))
    
plt.xlabel("offset") 
plt.ylabel("corrlation")
plt.plot(wave_offsets, corr_values)

In [ ]:
#序列相关性
def serial_corr(wave, lag=1):
    n = len(wave)
    y1 = wave.ys[lag:]
    y2 = wave.ys[:n-lag]
    corr = np.corrcoef(y1, y2)[0, 1]
    return corr

#  T/2  lag = T/2 * framerate = 1/2f * frametate 10000/2*440 = 11.363636363636363
print(serial_corr(wave1, lag=0)) # 自相关
print(serial_corr(wave1, lag=6)) # T/4 自相关性最小
print(serial_corr(wave1, lag=11)) 
print(serial_corr(wave1, lag=23))

In [ ]:
#自相关 求基频
import os
import sys
import thinkdsp

#print(sys.path)
#print(dir(thinkdsp))

thinkdsp_dir = "."
if thinkdsp.__file__:
    thinkdsp_dir = os.path.dirname(thinkdsp.__file__)

wave = thinkdsp.read_wave(thinkdsp_dir + '/28042__bcjordan__voicedownbew.wav')
wave.normalize()
wave.make_audio()

In [ ]:
#时域
wave.plot()

In [ ]:
#频域
import thinkplot
thinkplot.config(ylabel='Amp',
                 xlabel='Frequency (Hz)',
                 xlim=[0, 10000])

spectrum = wave.make_spectrum()
spectrum.plot()
#整个片段找不到明显的规律

In [ ]:
#整个片段的声谱图
import thinkplot
thinkplot.config(ylabel='Frequency(Hz)',
                 xlabel='Time (s)')
sg = wave.make_spectrogram(seg_length = int(0.01 * wave.framerate))
sg.plot()

In [ ]:
#较长的分段来提高频率的分辨率，但是音高随时间的变化就会变得模糊, 因为峰值(音高)会分布在这段波形的开始和结束的音高上

import thinkplot
thinkplot.config(xlabel='Time (s)',
                 ylabel='Frequency (Hz)',
                 ylim=[0, 5000]) #[200, 600]
spectrum = wave.make_spectrogram(int(0.01 * wave.framerate)) #较长的分段来提高频率的分辨率 0.01->100Hz(1/0.01)
spectrum.plot()

In [ ]:
thinkplot.config(xlabel='Time (s)',
                 ylabel='Frequency (Hz)',
                 ylim=[200, 600]) #
spectrum = wave.make_spectrogram(int(0.04 * wave.framerate)) #较长的分段来提高频率的分辨率 0.04->25Hz(1/0.01)
spectrum.plot()
# 在0.2s出基频大约在 425-450之间

In [ ]:
#通过相关系数验证基频
duration = 0.01
segment2 = wave.segment(start = 0.2023, duration = duration) #

print(np.corrcoef(segment.ys, segment2.ys))
print(segment.corr(segment2))
#周期 应该为 0.0023 -> 435Hz

In [ ]:
duration = 0.01
segment = wave.segment(start = 0.2, duration = duration)
thinkplot.config(ylabel='Amp',
                 xlabel='Frequency (Hz)',
                 xlim=[0, 5000])

spectrum = segment.make_spectrum()
print(spectrum.amps.argmax() * segment.framerate / len(segment.ys))
spectrum.plot()

In [ ]:
#细化上图信息，得到基频时400Hz, 与真实值有【0-50HZ】误差
duration = 0.01
segment = wave.segment(start = 0.2, duration = duration)
spectrum = segment.make_spectrum()
thinkplot.config(ylabel='Amp',
                 xlabel='Frequency (Hz)',
                 xlim=[0, 1000])
spectrum.plot()

In [ ]:
#通过相关计算基频

def serial_corr(wave, lag=1):
    n = len(wave)
    y1 = wave.ys[lag:]
    y2 = wave.ys[:n-lag]
    corr = np.corrcoef(y1, y2)[0, 1]
    return corr

def autocorr(wave):
    lags = range(len(wave.ys)//2)
    corrs = [serial_corr(wave, lag) for lag in lags]
    return lags, corrs

segment = wave.segment(start = 0.2, duration = duration)
lags, corrs = autocorr(segment)
thinkplot.plot(lags, corrs)

# 在偏移多少时，相关性最大，即正好为整数倍的周期
# 假设偏移N时，则 N/framerate = T, f = 1/T = framerate/N
off = 50 + np.argmax(corrs[50:150])
print(off) 
print("pitch:", segment.framerate/off) #周期跨越了101个采样数据点 T = 101 * (1/framerate) ->  f = 1/T = framerate/101

In [ ]:
#numpy 提供的相关系数计算
corr2 = np.correlate(segment.ys, segment.ys, mode='same') # -N/2 - N/2
corr2
N = int(len(corr2) / 2)
x = np.linspace(-N, N, len(corr2))
plt.plot(x, corr2)
plt.show()

#自相关函数是对称的，这是因为两个相同信号进行相关运算的时候， 正的lag值与负的lag值产生的影响是一样的。我们选择后半部分
half = corr2[N:]
plt.plot(half)
plt.show()

off = 50 + np.argmax(half[50:150])
print(off) 
print("pitch:", segment.framerate/off)